In [2]:
import pandas as pd
import torch
from torch import nn
#from torchsummary import summary
from collections import OrderedDict
from torchvision.models import resnet50, densenet121, inception_v3
from tqdm import tqdm

In [4]:
#from skimage.io import imread
import numpy as np
#from skimage.transform import resize
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image
from torchvision import models, transforms
import os
import cv2

ModuleNotFoundError: No module named 'cv2'

In [3]:
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [4]:
batch_size = 8

In [5]:
df_train = pd.read_csv('dataframe/train_fold"+str(i+1)+".csv')
df_val = pd.read_csv('dataframe/val_fold"+str(i+1)+".csv')
df_test= pd.read_csv('dataframe/test_fold"+str(i+1)+".csv')

In [6]:
def get_label(x):
    if x == 'yes':
        return 1
    else:
        return 0

In [7]:
df_train['label']=df_train['label'].apply(get_label)

In [8]:
df_val['label']=df_val['label'].apply(get_label)
df_test['label']=df_test['label'].apply(get_label)

In [9]:
num_class = len(df_train.label.unique())

In [10]:
class Classifier(nn.Module):
    def __init__(self, num_class):
        super().__init__()
        self.drop_out = nn.Dropout()
        self.linear = nn.Linear(2048, num_class)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.drop_out(x)
        x = self.linear(x)
        #x = torch.softmax(x, dim=-1)
        return x


class Backbone(nn.Module):
    def __init__(self):
        super().__init__()
        base_model = resnet50(pretrained=False)
        encoder_layers = list(base_model.children())
        self.backbone = nn.Sequential(*encoder_layers[:9])
                        
    def forward(self, x):
        return self.backbone(x)

In [18]:
layers = list(resnet50(pretrained=False).children())
layers[:9] == layers[:-1]

/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


True

In [20]:
list(inception_v3(pretrained=False).children())

/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


[BasicConv2d(
   (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
   (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 BasicConv2d(
   (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 MaxPo

In [21]:
list(resnet50(pretrained=False).children())

/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): Bottleneck(
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (downsample): Sequential(
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum

In [39]:
densenet121(pretrained=False).features

/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/var/home/mo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 128, ke

In [11]:
backbone = Backbone()
classifier = Classifier(num_class=num_class)

In [12]:
backbone.load_state_dict(torch.load("resnet50_torch.pt"))


<All keys matched successfully>

In [16]:
class createDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return self.dataframe.shape[0]
        
    def __getitem__(self, index):
        image = self.dataframe.iloc[index]["img_dir"]
        image = cv2.imread(image)
        image = (image-127.5)*2 / 255
        image = cv2.resize(image,(224,224))
        #image = np.transpose(image,(2,0,1))   
        if self.transform is not None:
            image = self.transform(image)
        label = self.dataframe.iloc[index]["label"]
        return {"image": image , "label": torch.tensor(label, dtype=torch.long)}

In [17]:
train_dataset = createDataset(df_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=20)
val_dataset = createDataset(df_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=20)

test_dataset = createDataset(df_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=20)

In [18]:
def train_model(model, criterion, optimizer,  num_epochs=30, fold_num):
    min_valid_loss = np.inf

    for e in range(num_epochs):
        train_loss = 0.0
        model.train()     # Optional when not using Model Specific layer
        for i_batch, info_batch in enumerate(train_loader):
            if torch.cuda.is_available():
                data, labels = info_batch['image'].to(device, dtype=torch.float), info_batch['label'].to(device)
            
            optimizer.zero_grad()
            target = model(data)
            loss = criterion(target,labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        valid_loss = 0.0
        model.eval()     # Optional when not using Model Specific layer
        for i_batch, info_batch in enumerate(val_loader):
            if torch.cuda.is_available():
                data, labels = info_batch['image'].to(device, dtype=torch.float), info_batch['label'].to(device)
            
            target = model(data)
            loss = criterion(target,labels)
            valid_loss = loss.item() * data.size(0)

        print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(val_loader)}')
        if min_valid_loss > valid_loss:
            print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
            min_valid_loss = valid_loss
            # Saving State Dict
            torch.save(model.state_dict(), 'acl_fold'+str(fold_num)+'_best_model.pth')
    return model

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [20]:
for f in range(5):
        ###use pretrain RIN-ResNet50 weights
        model = nn.Sequential(backbone, classifier)
        device = torch.device("cuda")
        model = model.to(device)
        best_model = train_model(model, criterion,optimizer, fold_num = f)

Epoch 1 		 Training Loss: 0.4457762529561808 		 Validation Loss: 0.31227233193137427
Validation Loss Decreased(inf--->3.434996) 	 Saving The Model
Epoch 2 		 Training Loss: 0.14679992552027926 		 Validation Loss: 0.09849888628179376
Validation Loss Decreased(3.434996--->1.083488) 	 Saving The Model
Epoch 3 		 Training Loss: 0.08154763455744475 		 Validation Loss: 0.12573414499109442
Epoch 4 		 Training Loss: 0.062108178586560904 		 Validation Loss: 0.25889845327897504
Epoch 5 		 Training Loss: 0.06391522991675679 		 Validation Loss: 0.017163275317712265
Validation Loss Decreased(1.083488--->0.188796) 	 Saving The Model
Epoch 6 		 Training Loss: 0.07031963857658051 		 Validation Loss: 0.03573957085609436
Epoch 7 		 Training Loss: 0.06533189754890582 		 Validation Loss: 0.18448266116055576
Epoch 8 		 Training Loss: 0.018094773113741643 		 Validation Loss: 0.0021281332116235385
Validation Loss Decreased(0.188796--->0.023409) 	 Saving The Model
Epoch 9 		 Training Loss: 0.01492005274361632